# Import packages

In [1]:
from model import *
from torchsummary import summary
from torchvision.models import efficientnet_b0
from torch.utils.data import DataLoader
from datasets import *
from torch.optim import lr_scheduler
from torchvision import transforms, datasets
import time
from torchvision.transforms import InterpolationMode

c:\Users\hoang\anaconda3\envs\edabk_violence_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\hoang\anaconda3\envs\edabk_violence_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\hoang\anaconda3\envs\edabk_violence_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)


# Import model

In [2]:
model = ViolenceEfficientNet()

summary(model, (45, 224, 224))

c:\Users\hoang\anaconda3\envs\edabk_violence_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\hoang\anaconda3\envs\edabk_violence_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 3, 224, 224]           1,218
            Conv2d-2         [-1, 32, 112, 112]             864
       BatchNorm2d-3         [-1, 32, 112, 112]              64
              SiLU-4         [-1, 32, 112, 112]               0
            Conv2d-5         [-1, 32, 112, 112]             288
       BatchNorm2d-6         [-1, 32, 112, 112]              64
              SiLU-7         [-1, 32, 112, 112]               0
 AdaptiveAvgPool2d-8             [-1, 32, 1, 1]               0
            Conv2d-9              [-1, 8, 1, 1]             264
             SiLU-10              [-1, 8, 1, 1]               0
           Conv2d-11             [-1, 32, 1, 1]             288
          Sigmoid-12             [-1, 32, 1, 1]               0
SqueezeExcitation-13         [-1, 32, 112, 112]               0
           Conv2d-14         [-1, 16, 1

# Constants

In [3]:
ROOT_DIR = "../../data"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

IMG_SIZE = (224, 224)

# Hyperparameters

In [4]:
epochs = 10
batch_size = 4

learning_rate = 1e-3


# Preparing dataset

In [5]:
splitDataset(train_ratio=0.8, dataset_root=ROOT_DIR)

Found empty or not enough images in seq folder, remove folder 2022_12_18_1671298532302602
Found empty or not enough images in seq folder, remove folder 2022_12_18_1671298657292724


### Standardlize process

In [6]:
def CalcStdVsMean():
    listImgs = []
    print("Calculating ...")
    for folder in os.listdir(ROOT_DIR):
        if folder != "dataset_info.csv":
            for fileName in os.listdir(f"{ROOT_DIR}/{folder}"):
                img = cv2.imread(f"{ROOT_DIR}/{folder}/{fileName}")
                listImgs.append(cv2.resize(img, IMG_SIZE, interpolation = cv2.INTER_AREA))
    listImgs = np.array(listImgs)
    return (listImgs/255.0).mean((0, 1, 2)), (listImgs /255.0).std((0,1,2))

MEAN, STD = CalcStdVsMean()

MEAN, STD

Calculating ...


(array([0.51551778, 0.43288471, 0.44265668]),
 array([0.19281362, 0.1960019 , 0.20439348]))

Can take MEAN, STD approximately as [0.5, 0.4, 0.4, 0.4], [0.19, 0.196, 0.2]

With the case using 45 channels => MEAN = MEAN *15, STD = STD *15

In [7]:
# MEAN, STD = list(MEAN), list(STD)

# MEAN = [MEAN[0], MEAN[1], [MEAN[2]]*15]
# STD = [STD[0], STD[1], [STD[2]]*15]

# MEAN, STD

MEAN, STD = list(MEAN)*15, list(STD)*15

In [8]:

data_transforms = transforms.Compose([
    MergeChannelTransForm(),
    transforms.ToTensor(),
    transforms.Resize(IMG_SIZE,interpolation=InterpolationMode.BICUBIC),
    transforms.Normalize(MEAN, STD)
])


train_dataset = Violence_Drone_Dataset(root_dir=ROOT_DIR,train=True, transform=data_transforms)
test_dataset = Violence_Drone_Dataset(root_dir=ROOT_DIR,train=False, transform=data_transforms)

train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=batch_size, drop_last=True)
test_loader = DataLoader(dataset= test_dataset, shuffle=False, batch_size=batch_size, drop_last=False)




Folder [100/100]: 100%|██████████| 100/100 [00:12<00:00,  7.95folder/s]


Found 100 data of type Train


Folder [26/26]: 100%|██████████| 26/26 [00:03<00:00,  8.08folder/s]

Found 26 data of type Test


In [9]:
samples = iter(train_loader)

imgs, labels = samples.__next__()

print(imgs.shape)

torch.Size([4, 45, 224, 224])


# Preparing for training

In [10]:
optimizer = torch.optim.AdamW(model.parameters(),
                              lr=learning_rate,
                              weight_decay=0.0005)

exp_lr_scheduler = lr_scheduler.MultiStepLR(
    optimizer,
    milestones=[20, 40, 60, 80, 100],
    gamma=0.2
)

criterion = nn.CrossEntropyLoss()

train_loss, train_accuracy = [], []
val_loss, val_accuracy = [], []


In [11]:
def fit(model, dataloader,epoch, epochs, train=True):
    if train:
        model.train()
    else:
        model.eval() 
    
    
    running_loss = 0.0
    running_correct = 0
    n_samples = 0
    
    print("Train" if train else "Val")
    
    with tqdm(dataloader, unit='batch') as tepoch:
        for images, labels in tepoch:    
            
            tepoch.set_description(f"Epoch [{epoch}/{epochs}]")
                
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct = (preds == labels).sum().item()

            running_correct += (preds == labels).sum().item()

            n_samples += labels.size(0)

            if train:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
            tepoch.set_postfix(loss=loss.item()/labels.size(0), accuracy=100.*correct/labels.size(0))
            
        process_loss = running_loss / n_samples
        process_acc = 100. * running_correct / n_samples
    
    return process_loss, process_acc
                

In [12]:
for images, labels in train_loader:
    print(labels)

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])


# Training and evaluating

In [14]:
model = model.to(device)

start = time.time()

for epoch in range(epochs):
    
    train_epoch_loss, train_epoch_acc = fit(model, train_loader, epoch, epochs)
    exp_lr_scheduler.step()
    
    val_epoch_loss, val_epoch_acc = fit(model, test_loader, epoch, epochs, train=False)
    
    train_loss.append(train_epoch_loss)
    train_accuracy.append(train_epoch_acc)
    
    val_loss.append(val_epoch_loss)
    val_accuracy.append(val_epoch_acc)

end = time.time()

print((end-start)/60, 'minutes')

torch.save(model.state_dict(), "last_weights.pth")
print("Model saved!")

Train


Epoch [0/10]: 100%|██████████| 25/25 [01:04<00:00,  2.57s/batch, accuracy=100, loss=0]      


Val


Epoch [0/10]: 100%|██████████| 7/7 [00:05<00:00,  1.28batch/s, accuracy=100, loss=0]      


Train


...

Train


Epoch [9/10]: 100%|██████████| 25/25 [00:38<00:00,  1.53s/batch, accuracy=100, loss=0]      


Val


Epoch [9/10]: 100%|██████████| 7/7 [00:05<00:00,  1.31batch/s, accuracy=100, loss=0]      

8.283403364817302 minutes
Model saved!
